# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
#Your code here
client_id = 'D83XSSMmMkyNHrMcSgSswA'
api_key = '5vOlw5YjRGlpHgS-U7KpuNsr3wxAU0OG_PtnHLdBNXe3GmVh8iChK4x1DDQJhZN4kgtDlFt7RMkgh95RCHYG6G72QM26MPLba0dJ1Zqdih-uY1PlDw4HlRDGLDoSXXYx'

term = 'italian'
location = "San Francisco"
SEARCH_LIMIT = 50

import requests

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
resp = requests.get(url, headers=headers, params=url_params)
resp.status_code == requests.codes.ok
print(resp)
print(type(resp.text))
print(resp.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "4KfQnlcSu4bbTqnvGdGptw", "alias": "beretta-san-francisco", "name": "Beretta", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/RsCeyBgtnXpmN9YTKEIvWg/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/beretta-san-francisco?adjust_creative=D83XSSMmMkyNHrMcSgSswA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=D83XSSMmMkyNHrMcSgSswA", "review_count": 3479, "categories": [{"alias": "italian", "title": "Italian"}, {"alias": "bars", "title": "Bars"}, {"alias": "pizza", "title": "Pizza"}], "rating": 4.0, "coordinates": {"latitude": 37.75384, "longitude": -122.42055}, "transactions": ["restaurant_reservation"], "price": "$$", "location": {"address1": "1199 Valencia St", "address2": "", "address3": "", "city": "San Francisco", "zip_code": "94110", "country": "US", "state": "CA", "display_address": ["1199 Valencia St", "San Francisco, CA 94110"]}, "phone": "+14156951199", "display_phone": "(415) 695-1199

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [2]:
import pandas as pd

In [3]:
type(resp.json()['total'])

int

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
alias            50 non-null object
categories       50 non-null object
coordinates      50 non-null object
display_phone    50 non-null object
distance         50 non-null float64
id               50 non-null object
image_url        50 non-null object
is_closed        50 non-null bool
location         50 non-null object
name             50 non-null object
phone            50 non-null object
price            49 non-null object
rating           50 non-null float64
review_count     50 non-null int64
transactions     50 non-null object
url              50 non-null object
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 6.0+ KB


In [5]:
df = pd.DataFrame(resp.json()['businesses'])
df.sort_values(by=['rating', 'name'])
df.head(10)

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,beretta-san-francisco,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 37.75384, 'longitude': -122.42055}",(415) 695-1199,1591.309745,4KfQnlcSu4bbTqnvGdGptw,https://s3-media1.fl.yelpcdn.com/bphoto/RsCeyB...,False,"{'address1': '1199 Valencia St', 'address2': '...",Beretta,+14156951199,$$,4.0,3479,[restaurant_reservation],https://www.yelp.com/biz/beretta-san-francisco...
1,sotto-mare-oysteria-and-seafood-san-francisco,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...","{'latitude': 37.79979, 'longitude': -122.40834}",(415) 398-3181,4980.880467,8dUaybEPHsZMgr1iKgqgMQ,https://s3-media2.fl.yelpcdn.com/bphoto/Efr_0z...,False,"{'address1': '552 Green St', 'address2': '', '...",Sotto Mare Oysteria & Seafood,+14153983181,$$,4.5,3499,[],https://www.yelp.com/biz/sotto-mare-oysteria-a...
2,bella-trattoria-san-francisco,"[{'alias': 'italian', 'title': 'Italian'}]","{'latitude': 37.78148, 'longitude': -122.46085}",(415) 221-0305,3133.710634,msT3LrLB4fhN04HYHuFsew,https://s3-media3.fl.yelpcdn.com/bphoto/QJZvt5...,False,"{'address1': '3854 Geary Blvd', 'address2': ''...",Bella Trattoria,+14152210305,$$,4.0,1349,[],https://www.yelp.com/biz/bella-trattoria-san-f...
3,the-italian-homemade-company-san-francisco-6,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 37.79742, 'longitude': -122.43097}",(415) 655-9325,4089.493554,FRpULkKmvD9caSKabQzq5w,https://s3-media2.fl.yelpcdn.com/bphoto/2yk0U7...,False,"{'address1': '1919 Union St', 'address2': '', ...",The Italian Homemade Company,+14156559325,$$,4.5,814,"[delivery, pickup]",https://www.yelp.com/biz/the-italian-homemade-...
4,roma-antica-san-francisco-3,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 37.79989, 'longitude': -122.4408}",(415) 896-4002,4351.770689,B09WOy0W83Od-Xw4xEXxog,https://s3-media1.fl.yelpcdn.com/bphoto/uPPwdS...,False,"{'address1': '3242 Scott St', 'address2': '', ...",Roma Antica,+14158964002,$$,4.5,450,"[delivery, pickup]",https://www.yelp.com/biz/roma-antica-san-franc...
5,a-mano-san-francisco-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 37.776963880538, 'longitude': -12...",(415) 506-7401,2100.777700,XhJ6ZYagcWNHpJ1H1KvNlQ,https://s3-media4.fl.yelpcdn.com/bphoto/ezG_zJ...,False,"{'address1': '450 Hayes St', 'address2': None,...",a Mano,+14155067401,$$,4.0,749,[],https://www.yelp.com/biz/a-mano-san-francisco-...
6,trattoria-da-vittorio-san-francisco-san-franci...,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 37.73933, 'longitude': -122.46775}",(415) 742-0300,3650.893099,FvPRM23d7NAOdyS_OX0MpQ,https://s3-media3.fl.yelpcdn.com/bphoto/EaCcOT...,False,"{'address1': '150 W Portal Ave', 'address2': N...",Trattoria da Vittorio - San Francisco,+14157420300,$$,4.0,666,"[delivery, pickup]",https://www.yelp.com/biz/trattoria-da-vittorio...
7,mona-lisa-restaurant-san-francisco,"[{'alias': 'italian', 'title': 'Italian'}]","{'latitude': 37.798352, 'longitude': -122.407728}",(415) 989-4917,4869.691300,bC3FfFqfaQu7gZznjkmt9Q,https://s3-media2.fl.yelpcdn.com/bphoto/LLD2tS...,False,"{'address1': '353 Columbus Ave', 'address2': '...",Mona Lisa Restaurant,+14159894917,$$,4.0,1445,"[restaurant_reservation, delivery, pickup]",https://www.yelp.com/biz/mona-lisa-restaurant-...
8,spiazzo-ristorante-san-francisco,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 37.74035, 'longitude': -122.46588}",(415) 664-9511,3456.701479,NWu1cDtYArkjWw2F8PzU9Q,https://s3-media2.fl.yelpcdn.com/bphoto/67HuoK...,False,"{'address1': '33 W Portal Ave', 'address2': ''...",Spiazzo Ristorante,+14156649511,$$,4.0,634,"[restaurant_reservation, pickup]",https://www.yelp.com/biz/spiazzo-ristorante-sa...
9,the-italian-homemade-company-san-francisco-8,"[{'alias': 'italian', 'title': 'Italian'}, {'a...","{'latitude': 37.801752, 'longitude': -122.411851}",(415) 712-8874,5030.6731

In [23]:
# Your code here; use a function or loop to retrieve all the results from your original request
def all_results(url_params, api_key):
    num = resp.json()['total']
    cur = 0
    dfs = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        dfs.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    df = pd.concat(dfs, ignore_index=True)
    return df
    





In [24]:
all_results(url_params, api_key)

NameError: name 'yelp_call' is not defined

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
#Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!